In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sns

In [2]:
census_df=pd.read_csv('census.csv')
census_df['COUNTYFP10']=census_df['COUNTYFP10'].astype(str).str.pad(3,fillchar='0')
census_df.head()

,county_id,county,population,COUNTYFP10,latitude,longitude
0,0,Adair,7496,001,41.328528,-94.478164
1,1,Adams,3704,003,41.021656,-94.696906
2,2,Allamakee,14061,005,43.274964,-91.382751
3,3,Appanoose,12317,007,40.744683,-92.870345
4,4,Audubon,5674,009,41.679178,-94.904312


In [4]:
np.zeros(10,dtype=int)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [10]:
data={
    'county_id':[0,1,2,3,4,5,6,7,8,9],
    'district':[0,1,2,3,0,1,2,3,2,3],
    'population':[20,30,20,30,40,20,30,20,30,40],
    'num_switches':np.zeros(10,dtype=int)
}
df=pd.DataFrame(data)
df

,county_id,district,population,num_switches
0,0,0,20,0
1,1,1,30,0
2,2,2,20,0
3,3,3,30,0
4,4,0,40,0
5,5,1,20,0
6,6,2,30,0
7,7,3,20,0
8,8,2,30,0
9,9,3,40,0


In [27]:
districts=[]
for i in range(4):
    districts.append(df['population'].loc[df['district']==i].sum())
    print(districts)

[60]
[60, 50]
[60, 50, 80]
[60, 50, 80, 90]


In [38]:
for i in range(4):
    print(    max(df['population'].loc[df['district']==i]))

40
30
30
40


In [54]:
df['population'].loc[df['district']==3]==max(df['population'].loc[df['district']==3])

3    False
7    False
9     True
Name: population, dtype: bool

In [106]:
for i in range(4):
    district_max_pop=max(df['population'].loc[df['district']==i])
    district_min_switches=min(df['num_switches'].loc[df['district']==i])
    district=df.loc[df['district']==i]
    # print(district)
    for j in range(district.shape[0]):
        if (district.iloc[j][2]==district_max_pop) & (district.iloc[j][3]==district_min_switches):
            print(district.iloc[j][0])
        # print(i,district.iloc[j][2])

4
1
5
6
8
9
